In [ ]:
!wget https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/reviews.txt

--2022-07-24 14:01:32--  https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt.3’

reviews.txt.3       100%[===================>]  32.12M   155MB/s    in 0.2s    

2022-07-24 14:01:33 (155 MB/s) - ‘reviews.txt.3’ saved [33678267/33678267]



In [ ]:
!wget https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/labels.txt 

--2022-07-24 14:01:33--  https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt.3’

labels.txt.3        100%[===================>] 219.73K  --.-KB/s    in 0.02s   

2022-07-24 14:01:33 (8.61 MB/s) - ‘labels.txt.3’ saved [225000/225000]



In [ ]:
# Import dependencies
from string import punctuation
from collections import Counter
import numpy as np
import pandas as pd

In [ ]:
# Read data from text files
with open("/content/reviews.txt") as f:
  reviews = f.read()

with open("/content/labels.txt") as f:
  labels = f.read()

# Data Exploration and Preprocessing

In [ ]:
reviews[: 1000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

In [ ]:
# Characters present in reviews 
set(reviews)

{'\n',
 ' ',
 '.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
labels[: 1000]

'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\

In [ ]:
# Convert alphabets to lowercase
text = reviews.lower()

# Remove the punctuations
text = ''.join([char  for char in text  if char not in punctuation])

# Remove '\n' character
reviews_split = text.split('\n')
text = ' '.join(reviews_split)

text[: 5000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [ ]:
labels = labels.split()
labels[: 10]

['positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [ ]:
# Check if zero-length reviews are present
review_lengths = Counter(len(review)  for review in reviews_split)
print(f"Zero-length reviews: {review_lengths[0]}")

Zero-length reviews: 1


In [ ]:
# Get index of reviews having non-zero length
non_zero_idxs = [idx  for idx, review in enumerate(reviews_split)  if len(review)!=0]

# Remove zero-length reviews and their corresponding labels
reviews_split = [reviews_split[idx]  for idx in non_zero_idxs]
labels = [labels[idx]  for idx in non_zero_idxs]

In [ ]:
print(f"No. of reviews: {len(reviews_split)}")
print(f"No. of labels : {len(labels)}")

In [ ]:
# Get a list of words used in reviews
words = text.split()
print(words[: 15])
print(f"\nThere are {len(words)} words in reviews.")

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other']

There are 6020196 words in reviews.


In [ ]:
# Get unique words
unique_words = set(words)
print(f"There are {len(unique_words)} unique words in reviews.")

There are 74072 unique words in reviews.


### Create Dictionary

In [ ]:
int_vocab = dict(enumerate(unique_words))

dict(list(int_vocab.items())[:10])

{0: 'sensational',
 1: 'positions',
 2: 'mishin',
 3: 'aroo',
 4: 'brannon',
 5: 'hulking',
 6: 'holfernes',
 7: 'scooping',
 8: 'blackmailing',
 9: 'feux'}

In [ ]:
vocab_int = {int_vocab[i]: i  for i in int_vocab}

dict(list(vocab_int.items())[:10])

{'aroo': 3,
 'blackmailing': 8,
 'brannon': 4,
 'feux': 9,
 'holfernes': 6,
 'hulking': 5,
 'mishin': 2,
 'positions': 1,
 'scooping': 7,
 'sensational': 0}

### Tokenize Reviews

In [ ]:
reviews_int = []

for review in reviews_split:
  reviews_int.append([vocab_int[word]  for word in review.split()])

In [ ]:
# Get length of the longest review
max_len = max(len(review)  for review in reviews_int)

# Get the average length of reviews
avg_len = sum([len(review)  for review in reviews_int]) / len(reviews_int)

print(f"The length of the longest review is {max_len}.")
print(f"The average length of reviews is {avg_len:.2f}")

The length of the longest review is 2514.
The average length of reviews is 240.81


In [ ]:
def pad_reviews_with_zeros(reviews_int, seq_len):
  tokenized_reviews = np.zeros((len(reviews_int), seq_len), dtype=np.int32)

  for idx, review in enumerate(reviews_int):
    tokenized_reviews[idx][:len(review)] = np.array(review)[:seq_len] 

  return tokenized_reviews

In [ ]:
tokenized_reviews = pad_reviews_with_zeros(reviews_int, seq_len=512)

In [ ]:
a = np.expand_dims(tokenized_reviews, axis=0)
a.shape

(1, 25000, 512)

In [ ]:
labels = np.array([1 if label=="positive" else 0  for label in labels])
labels[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
tokenized_reviews[:5]

array([[70813, 16987, 54502, ...,     0,     0,     0],
       [32970, 12694, 64516, ...,     0,     0,     0],
       [26854, 36487, 57791, ...,     0,     0,     0],
       [58899, 51135, 45413, ..., 45413, 43515, 22207],
       [25052, 62232, 38777, ...,     0,     0,     0]], dtype=int32)

In [ ]:
# df = pd.DataFrame(data=(tokenized_reviews, labels), index=["tokenized_reviews", "labels"]).T
# df.head()
data = pd.DataFrame((tokenized_reviews, labels), index=["Reviews", "Labels"]).T

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


ValueError: ignored

In [ ]:
# tokenized_reviews = [[12, 13, 15],
#                      [45, 85, 64, 45, 63, 78, 45, 85, 64, 45, 63, 78],
#                      [16, 75, 96, 42]]